# 🏥 Doctis AI - Fine-Tuning Notebook

**Projet:** Doctis AI - Assistant de Pré-diagnostic Médical

**Auteurs:** Adam Beloucif & Amina Medjdoub

**Module:** IA Générative & Data Engineering - EFREI 2025

---

Ce notebook permet de :
1. Fine-tuner un SLM (Phi-3 / TinyLlama) sur des données médicales
2. Convertir le modèle en format GGUF pour exécution CPU
3. Préparer le déploiement sur Render

## 📦 Installation des dépendances

In [ ]:
# =============================================================================
# Projet: Doctis AI
# Auteurs: Adam Beloucif & Amina Medjdoub
# =============================================================================

!pip install -q transformers==4.38.0 datasets accelerate peft bitsandbytes trl
!pip install -q huggingface_hub sentencepiece

print("✅ Dépendances installées")

In [ ]:
# Connexion à Hugging Face (optionnel mais recommandé)
from huggingface_hub import login

# Décommentez et ajoutez votre token HF
# login(token="hf_VOTRE_TOKEN_ICI")

print("💡 Conseil: Ajoutez votre token HF pour accéder aux modèles privés")

## 🔧 Configuration du modèle

In [ ]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer
from datasets import load_dataset

# =============================================================================
# CONFIGURATION - Modifiez selon vos besoins
# =============================================================================

# Choix du modèle de base
MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"  # Léger, rapide
# MODEL_NAME = "microsoft/Phi-3-mini-4k-instruct"  # Plus performant

OUTPUT_DIR = "./doctis-ai-finetuned"

print(f"📌 Modèle sélectionné: {MODEL_NAME}")
print(f"📁 Répertoire de sortie: {OUTPUT_DIR}")

In [ ]:
# Configuration de la quantification 4-bit
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

# Chargement du modèle
print(f"⏳ Chargement de {MODEL_NAME}...")

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

print(f"✅ Modèle chargé avec succès!")
print(f"   GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'Non disponible'}")

## 🎯 Configuration LoRA

In [ ]:
# Configuration LoRA pour un fine-tuning efficace
lora_config = LoraConfig(
    r=16,                       # Rang des matrices LoRA
    lora_alpha=32,              # Facteur d'échelle
    target_modules=[            # Modules ciblés
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Préparation du modèle pour l'entraînement
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

# Affichage des statistiques
model.print_trainable_parameters()

## 📊 Chargement du Dataset

In [ ]:
# =============================================================================
# OPTION 1: Dataset ChatDoctor (recommandé pour le médical)
# =============================================================================

print("⏳ Chargement du dataset...")

# Charge un sous-ensemble pour l'entraînement rapide
dataset = load_dataset(
    "lavita/ChatDoctor-HealthCareMagic-100k",
    split="train[:3000]"  # Ajustez selon vos besoins
)

print(f"✅ Dataset chargé: {len(dataset)} exemples")
print(f"   Colonnes: {dataset.column_names}")

In [ ]:
# Formatage des données pour l'entraînement
def format_medical_prompt(example):
    """Formate les exemples au format chat médical."""

    # Extraction des champs (adapté au format ChatDoctor)
    instruction = example.get("instruction", example.get("input", ""))
    response = example.get("output", example.get("response", ""))

    # Format de prompt pour Doctis AI
    formatted_text = f"""<|system|>
Tu es Doctis AI, un assistant médical bienveillant et professionnel.
Tu analyses les symptômes décrits par le patient et fournis des informations
médicales claires, rassurantes et fondées sur des connaissances médicales.
Tu rappelles toujours l'importance de consulter un professionnel de santé.<|end|>
<|user|>
{instruction}<|end|>
<|assistant|>
{response}<|end|>"""

    return {"text": formatted_text}

# Application du formatage
dataset = dataset.map(format_medical_prompt)

# Aperçu d'un exemple
print("📝 Exemple de données formatées:")
print("-" * 50)
print(dataset[0]["text"][:500] + "...")

## 🏋️ Entraînement

In [ ]:
# Configuration de l'entraînement
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=3,                    # Nombre d'époques
    per_device_train_batch_size=4,         # Batch size
    gradient_accumulation_steps=4,         # Accumulation de gradients
    learning_rate=2e-4,                    # Taux d'apprentissage
    weight_decay=0.01,                     # Régularisation
    warmup_ratio=0.03,                     # Warmup
    lr_scheduler_type="cosine",            # Scheduler
    logging_steps=25,                      # Fréquence de logging
    save_steps=500,                        # Fréquence de sauvegarde
    save_total_limit=2,                    # Nombre de checkpoints max
    fp16=True,                             # Mixed precision
    optim="paged_adamw_8bit",              # Optimiseur efficace
    report_to="none",                      # Pas de logging externe
    max_grad_norm=0.3,                     # Gradient clipping
)

# Création du trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    tokenizer=tokenizer,
    args=training_args,
    dataset_text_field="text",
    max_seq_length=512,
    packing=False
)

print("✅ Trainer configuré")
print(f"   Époques: {training_args.num_train_epochs}")
print(f"   Batch size effectif: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")

In [ ]:
# 🚀 LANCEMENT DE L'ENTRAÎNEMENT
print("="*60)
print("🏥 DOCTIS AI - Démarrage du fine-tuning")
print("   Auteurs: Adam Beloucif & Amina Medjdoub")
print("="*60)

trainer.train()

print("\n" + "="*60)
print("✅ Entraînement terminé!")
print("="*60)

In [ ]:
# Sauvegarde du modèle fine-tuné
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

print(f"✅ Modèle sauvegardé dans {OUTPUT_DIR}")

## 🔀 Fusion des poids LoRA

In [ ]:
from peft import AutoPeftModelForCausalLM

print("⏳ Fusion des poids LoRA...")

# Recharge le modèle avec les adaptateurs
model = AutoPeftModelForCausalLM.from_pretrained(
    OUTPUT_DIR,
    device_map="auto",
    torch_dtype=torch.float16
)

# Fusion des poids
merged_model = model.merge_and_unload()

# Sauvegarde du modèle fusionné
MERGED_DIR = "./doctis-ai-merged"
merged_model.save_pretrained(MERGED_DIR, safe_serialization=True)
tokenizer.save_pretrained(MERGED_DIR)

print(f"✅ Modèle fusionné sauvegardé dans {MERGED_DIR}")

## 📦 Conversion en GGUF

In [ ]:
# Installation de llama.cpp
!git clone https://github.com/ggerganov/llama.cpp
%cd llama.cpp
!pip install -q -r requirements.txt

print("✅ llama.cpp installé")

In [ ]:
# Conversion au format GGUF
print("⏳ Conversion en GGUF (format F16)...")

!python convert_hf_to_gguf.py ../doctis-ai-merged \
    --outfile ../doctis-ai-f16.gguf \
    --outtype f16

print("✅ Conversion F16 terminée")

In [ ]:
# Compilation de llama.cpp pour la quantification
!make -j llama-quantize

print("✅ llama-quantize compilé")

In [ ]:
# Quantification Q4_K_M (recommandée pour CPU)
print("⏳ Quantification Q4_K_M...")

!./llama-quantize ../doctis-ai-f16.gguf ../doctis-ai-Q4_K_M.gguf Q4_K_M

print("✅ Quantification terminée")

# Affiche les tailles
!ls -lh ../doctis-ai-*.gguf

## 📥 Téléchargement du modèle

In [ ]:
from google.colab import files

print("📥 Téléchargement du modèle quantifié...")
print("   Cela peut prendre quelques minutes selon la taille du fichier.")

files.download("../doctis-ai-Q4_K_M.gguf")

print("\n" + "="*60)
print("🎉 FÉLICITATIONS!")
print("   Votre modèle Doctis AI est prêt pour le déploiement!")
print("   ")
print("   Prochaine étape: Placez le fichier .gguf dans")
print("   le dossier backend/models/ de votre projet.")
print("="*60)
print("\n   Projet: Doctis AI")
print("   Auteurs: Adam Beloucif & Amina Medjdoub")
print("   EFREI 2025")

## 🧪 Test du modèle (Optionnel)

In [ ]:
# Test rapide du modèle GGUF avec llama-cpp-python
!pip install -q llama-cpp-python

from llama_cpp import Llama

print("⏳ Chargement du modèle GGUF pour test...")

llm = Llama(
    model_path="../doctis-ai-Q4_K_M.gguf",
    n_ctx=512,
    n_threads=4,
    verbose=False
)

# Test de génération
test_prompt = """<|system|>
Tu es Doctis AI, un assistant médical bienveillant.<|end|>
<|user|>
J'ai mal au ventre en bas à droite depuis ce matin, avec des nausées.<|end|>
<|assistant|>
"""

print("\n📝 Test de génération:")
print("-" * 50)

response = llm(
    test_prompt,
    max_tokens=200,
    temperature=0.7,
    stop=["<|end|>", "<|user|>"]
)

print(response["choices"][0]["text"])
print("-" * 50)
print("✅ Test réussi!")